## ANALISA BENCHMARK PER JENIS ALAT BERAT (FILE PAK ABDI)

In [11]:
import pandas as pd
import numpy as np
import os
import re
import warnings
import gc

warnings.filterwarnings('ignore')

# ==============================================================================
# FUNGSI PEMBERSIH NAMA UNIT (GLOBAL)
# ==============================================================================
def clean_unit_name(name):
    if pd.isna(name): return ""
    name = str(name).upper().strip()
    
    # 1. Standardisasi Typo (Samakan FORKLIFT vs FORKLIF)
    name = name.replace("FORKLIFT", "FORKLIF")
    
    # 2. Hapus semua simbol (/, ., -, spasi) -> Hanya Huruf & Angka
    # Contoh: "L 9902 UR / S75" -> "L9902URS75"
    name = re.sub(r'[^A-Z0-9]', '', name)
    
    return name

# ==============================================================================
# BAGIAN 1: LOAD MASTER DATA & GROUPING STRATEGY
# ==============================================================================
print("🚀 [1/3] MEMBACA & MEMBERSIHKAN MASTER DATA ALAT BERAT...")

master_file = 'cost & bbm 2022 sd 2025.xlsx'
master_data_map = {} 
master_keys_set = set()

try:
    if os.path.exists(master_file):
        df_map = pd.read_excel(master_file, sheet_name='Sheet2', header=1)
        
        # Rename kolom agar standar
        if 'NAMA ALAT BERAT' in df_map.columns:
            df_map.rename(columns={
                'NAMA ALAT BERAT': 'Unit_Original',
                'DES 2025': 'Lokasi',
                'ALAT BERAT': 'Jenis_Alat',
                'CAP': 'Capacity'
            }, inplace=True)
        
        # Filter Kolom Penting
        df_map = df_map[['Unit_Original', 'Lokasi', 'Jenis_Alat', 'Capacity']].copy()
        df_map.dropna(subset=['Unit_Original'], inplace=True)
        
        # --- BUAT CLEAN NAME (ID) ---
        df_map['Unit_ID'] = df_map['Unit_Original'].apply(clean_unit_name)
        
        # Hapus "DUMMY"
        df_map = df_map[~df_map['Unit_Original'].astype(str).str.upper().str.contains('DUMMY', na=False)]
        
        # --- PERBAIKAN MANUAL: GANTI CAPACITY L 9025 US JADI 40 ---
        # Cari unit L 9025 US (dengan clean ID)
        target_unit_id = clean_unit_name("L 9025 US")
        # Update kapasitasnya menjadi 40
        df_map.loc[df_map['Unit_ID'] == target_unit_id, 'Capacity'] = 40
        
        # Pembulatan Kapasitas
        df_map['Capacity'] = pd.to_numeric(df_map['Capacity'], errors='coerce').fillna(0)
        df_map['Capacity'] = np.floor(df_map['Capacity'] + 0.5).astype(int)
        
        # --- ATURAN: HAPUS KAPASITAS 0 ---
        df_map = df_map[df_map['Capacity'] > 0]
        
        print(f"   -> Data Master Terbaca: {len(df_map)} unit valid (Capacity > 0)")

        # --- STRATEGI GROUPING ---
        def determine_group(row):
            jenis = str(row['Jenis_Alat']).upper().strip()
            cap = row['Capacity']
            
            if jenis in ['FORKLIFT', 'SIDE LOADER']:
                if cap < 5: return f"{jenis} (< 5T)"
                elif cap < 10: return f"{jenis} (5-9T)"
                elif cap < 15: return f"{jenis} (10-14T)"
                else: return f"{jenis} (>= 15T)"
            elif 'CRANE' in jenis:
                if cap < 100: return "CRANE (< 100T)"
                else: return "CRANE (>= 100T)"
            elif jenis in ['REACH STACKER', 'TOP LOADER']:
                if cap > 35: return f"{jenis} (> 35T)"
                else: return f"{jenis} (<= 35T)"
            else:
                return f"{jenis} ({cap}T)"

        df_map['Benchmark_Group'] = df_map.apply(determine_group, axis=1)
        
        # SIMPAN KE DICTIONARY
        for _, row in df_map.iterrows():
            clean_id = row['Unit_ID']
            if clean_id:
                master_data_map[clean_id] = {
                    'Unit_Name': row['Unit_Original'],
                    'Benchmark_Group': row['Benchmark_Group'],
                    'Lokasi': row['Lokasi'],
                    'Jenis_Alat': row['Jenis_Alat'],
                    'Capacity': row['Capacity']
                }
                master_keys_set.add(clean_id)
            
        print(f"✅ Mapping Selesai. {len(master_data_map)} unit siap dicocokkan.")
        
    else:
        print(f"❌ File '{master_file}' tidak ditemukan.")

except Exception as e:
    print(f"❌ Error membaca file Master: {e}")

🚀 [1/3] MEMBACA & MEMBERSIHKAN MASTER DATA ALAT BERAT...
   -> Data Master Terbaca: 241 unit valid (Capacity > 0)
✅ Mapping Selesai. 241 unit siap dicocokkan.


In [12]:
# ==============================================================================
# BAGIAN 2: BACA EXCEL TRANSAKSI & SUPER SMART MATCHING
# ==============================================================================
print("\n🚀 [2/3] MEMPROSES FILE TRANSAKSI 'BBM AAB.xlsx'...")

filename_excel = 'BBM AAB.xlsx'
target_sheets = ['JAN', 'FEB', 'MAR', 'APR', 'MEI', 'JUN', 'JUL', 'AGT', 'SEP', 'OKT', 'NOV']
raw_data_list = []
matched_units_count = set()

if os.path.exists(filename_excel) and master_data_map:
    try:
        xls = pd.ExcelFile(filename_excel)
        
        for sheet in target_sheets:
            if sheet in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet, header=None)
                
                # Parsing Header
                unit_names_row = df.iloc[0].ffill()
                headers = df.iloc[2]
                dates = df.iloc[3:, 0]
                
                for col in range(1, df.shape[1]):
                    header_str = str(headers[col]).strip().upper()
                    
                    if header_str in ['HM', 'LITER', 'KELUAR', 'PEMAKAIAN']:
                        
                        raw_unit_name = str(unit_names_row[col]).strip().upper()
                        
                        # --- SYARAT 1: FILTER AWAL & EXCLUDE ---
                        if raw_unit_name == "" or "UNNAMED" in raw_unit_name or "EQUIP NAME" in raw_unit_name or "TOTAL" in raw_unit_name:
                            continue
                        
                        # Exclude Unit Tertentu
                        if raw_unit_name.startswith(('GENSET', 'KOMPRESSOR', 'MESIN', 'TANGKI', 'SPBU', 'MOBIL')):
                            continue
                        
                        # Bersihkan Nama
                        clean_trx_id = clean_unit_name(raw_unit_name)
                        matched_id = None
                        
                        # --- SYARAT 2: MANUAL MAPPING (PRIORITAS UTAMA) ---
                        if "FL RENTAL 01" in raw_unit_name and "TIMIKA" not in raw_unit_name:
                            temp_id = clean_unit_name("FL RENTAL 01 TIMIKA")
                            if temp_id in master_data_map: matched_id = temp_id
                        
                        elif "TOBATI" in raw_unit_name and "KALMAR 32T" in raw_unit_name:
                            temp_id = clean_unit_name("TOP LOADER KALMAR 35T/TOBATI") 
                            if temp_id in master_data_map: matched_id = temp_id
                            
                        elif "L 8477 UUC" in raw_unit_name: # Handle L 8477 (EX. L 9902 UR)
                            temp_id = clean_unit_name("L 9902 UR / S75")
                            if temp_id in master_data_map: matched_id = temp_id

                        # --- LOGIKA MATCHING OTOMATIS ---
                        
                        # A. Cek Langsung (Exact Match)
                        if not matched_id and clean_trx_id in master_data_map:
                            matched_id = clean_trx_id
                        
                        # B. Cek "EX." (SYARAT 3)
                        if not matched_id and "EX." in raw_unit_name:
                            try:
                                part_after_ex = raw_unit_name.split("EX.")[-1].replace(")", "").strip()
                                clean_after_ex = clean_unit_name(part_after_ex)
                                
                                # Cek Exact Match
                                if clean_after_ex in master_data_map:
                                    matched_id = clean_after_ex
                                # Cek Partial Match (Jika nama di Master lebih panjang, misal ada "/ S75")
                                elif clean_after_ex:
                                    for m_key in master_keys_set:
                                        if clean_after_ex in m_key:
                                            matched_id = m_key
                                            break
                            except: pass

                        # C. Cek Sebelum Kurung " (" (SYARAT 4)
                        if not matched_id and " (" in raw_unit_name:
                            try:
                                part_before = raw_unit_name.split(" (")[0].strip()
                                clean_before = clean_unit_name(part_before)
                                if clean_before in master_data_map:
                                    matched_id = clean_before
                            except: pass

                        # --- JIKA MATCH, AMBIL DATANYA ---
                        if matched_id:
                            matched_units_count.add(matched_id)
                            metric_type = 'HM' if header_str == 'HM' else 'LITER'
                            vals = pd.to_numeric(df.iloc[3:, col], errors='coerce')
                            
                            info_unit = master_data_map[matched_id]
                            
                            temp_df = pd.DataFrame({
                                'Date': dates,
                                'Unit_Name': info_unit['Unit_Name'], # Pakai nama Master
                                'Benchmark_Group': info_unit['Benchmark_Group'],
                                'Lokasi': info_unit['Lokasi'],
                                'Jenis_Alat': info_unit['Jenis_Alat'],
                                'Capacity': info_unit['Capacity'],
                                'Metric': metric_type,
                                'Value': vals
                            })
                            
                            temp_df.dropna(subset=['Value', 'Date'], inplace=True)
                            if not temp_df.empty:
                                raw_data_list.append(temp_df)
                
                del df; gc.collect()
                
    except Exception as e:
        print(f"   ❌ Error processing Excel: {e}")

print(f"✅ Data Transaksi Selesai. {len(matched_units_count)} unit unik berhasil dicocokkan.")


🚀 [2/3] MEMPROSES FILE TRANSAKSI 'BBM AAB.xlsx'...
✅ Data Transaksi Selesai. 236 unit unik berhasil dicocokkan.


In [13]:
# ==============================================================================
# BAGIAN 3: PIVOT, DELTA HM, ANALISA & EXPORT
# ==============================================================================
print("\n🚀 [3/3] MENGHITUNG BENCHMARK & MEMBUAT LAPORAN...")

unit_perf_data = []

if raw_data_list:
    df_all = pd.concat(raw_data_list, ignore_index=True)
    df_all['Date'] = pd.to_datetime(df_all['Date'], dayfirst=True, errors='coerce')
    df_all.dropna(subset=['Date'], inplace=True)
    
    # Pivot
    df_pivot = df_all.pivot_table(
        index=['Unit_Name', 'Benchmark_Group', 'Lokasi', 'Jenis_Alat', 'Capacity', 'Date'],
        columns='Metric',
        values='Value',
        aggfunc='sum'
    ).reset_index()
    
    if 'HM' not in df_pivot.columns: df_pivot['HM'] = 0
    if 'LITER' not in df_pivot.columns: df_pivot['LITER'] = 0
    df_pivot['HM'] = df_pivot['HM'].fillna(0)
    df_pivot['LITER'] = df_pivot['LITER'].fillna(0)
    
    # Sortir Wajib
    df_pivot.sort_values(by=['Unit_Name', 'Date'], inplace=True)
    
    # --- HITUNG JAM KERJA (DELTA HM) ---
    df_pivot['HM_Clean'] = df_pivot['HM'].replace(0, np.nan)
    df_pivot['HM_Clean'] = df_pivot.groupby('Unit_Name')['HM_Clean'].ffill()
    df_pivot['HM_Clean'] = df_pivot['HM_Clean'].fillna(0)
    
    df_pivot['Delta_HM'] = df_pivot.groupby('Unit_Name')['HM_Clean'].diff().fillna(0)
    
    # Cleaning Delta
    df_pivot.loc[df_pivot['Delta_HM'] < 0, 'Delta_HM'] = 0 
    df_pivot.loc[df_pivot['Delta_HM'] > 100, 'Delta_HM'] = 0 
    
    # Summary per Unit
    final_stats = df_pivot.groupby(['Unit_Name', 'Benchmark_Group', 'Lokasi', 'Jenis_Alat', 'Capacity']).agg({
        'LITER': 'sum',
        'Delta_HM': 'sum'
    }).reset_index()
    
    final_stats.rename(columns={'LITER': 'Total_Liter', 'Delta_HM': 'Total_HM_Work'}, inplace=True)
    unit_perf_data = final_stats.to_dict('records')

# --- EXPORT ---
if unit_perf_data:
    df_raw = pd.DataFrame(unit_perf_data)
    
    # Ratio
    df_raw['Fuel_Ratio'] = df_raw.apply(lambda row: row['Total_Liter'] / row['Total_HM_Work'] if row['Total_HM_Work'] > 0 else 0, axis=1)
    
    # Benchmark Median
    df_valid = df_raw[(df_raw['Total_HM_Work'] > 0) & (df_raw['Total_Liter'] > 0)].copy()
    benchmark_stats = df_valid.groupby('Benchmark_Group')['Fuel_Ratio'].median().reset_index()
    benchmark_stats.rename(columns={'Fuel_Ratio': 'Group_Benchmark_Median'}, inplace=True)
    
    # Merge
    df_final = pd.merge(df_raw, benchmark_stats, on='Benchmark_Group', how='left')
    
    # Status
    def get_status(row):
        if row['Total_HM_Work'] <= 0 or row['Total_Liter'] <= 0: 
            return "DATA TIDAK LENGKAP / INAKTIF"
        bench = row['Group_Benchmark_Median']
        if pd.isna(bench): return "SINGLE UNIT (NO COMPARISON)"
        return "EFISIEN" if row['Fuel_Ratio'] <= bench else "BOROS"

    df_final['Performance_Status'] = df_final.apply(get_status, axis=1)
    
    # Potensi Pemborosan
    df_final['Potensi_Pemborosan_Liter'] = 0
    boros_mask = df_final['Performance_Status'] == "BOROS"
    df_final.loc[boros_mask, 'Potensi_Pemborosan_Liter'] = (
        (df_final.loc[boros_mask, 'Fuel_Ratio'] - df_final.loc[boros_mask, 'Group_Benchmark_Median']) 
        * df_final.loc[boros_mask, 'Total_HM_Work']
    )
    
    # Formatting
    df_final[['Fuel_Ratio', 'Group_Benchmark_Median']] = df_final[['Fuel_Ratio', 'Group_Benchmark_Median']].round(2)
    df_final['Potensi_Pemborosan_Liter'] = df_final['Potensi_Pemborosan_Liter'].round(0)
    
    df_active = df_final[df_final['Performance_Status'] != "DATA TIDAK LENGKAP / INAKTIF"].copy()
    df_inactive = df_final[df_final['Performance_Status'] == "DATA TIDAK LENGKAP / INAKTIF"].copy()
    
    df_active.sort_values(['Benchmark_Group', 'Fuel_Ratio'], inplace=True)

    output_file = 'Benchmark_Per_Alat_Berat_Data_Baru2.xlsx'
    try:
        with pd.ExcelWriter(output_file) as writer:
            cols = ['Benchmark_Group', 'Jenis_Alat', 'Capacity', 'Lokasi', 'Unit_Name', 
                    'Total_Liter', 'Total_HM_Work', 'Fuel_Ratio', 
                    'Group_Benchmark_Median', 'Performance_Status', 'Potensi_Pemborosan_Liter']
            df_active[cols].to_excel(writer, sheet_name='Rapor_Unit_Aktif', index=False)
            
            summary_grp = df_active.groupby('Benchmark_Group').agg({
                'Unit_Name': 'count',
                'Total_Liter': 'sum',
                'Total_HM_Work': 'sum',
                'Group_Benchmark_Median': 'first',
                'Potensi_Pemborosan_Liter': 'sum'
            }).reset_index().rename(columns={'Unit_Name': 'Populasi_Unit'})
            summary_grp.to_excel(writer, sheet_name='Summary_Group', index=False)
            
            df_inactive[['Benchmark_Group', 'Jenis_Alat', 'Unit_Name', 'Total_Liter', 'Total_HM_Work']].to_excel(writer, sheet_name='Unit_Inaktif', index=False)
            
        print(f"\n💾  SUKSES! Laporan Tersimpan: '{output_file}'")
        print(f"    - Unit Aktif: {len(df_active)}")
        print(f"    - Unit Inaktif: {len(df_inactive)}")

    except Exception as e:
        print(f"⚠️ Gagal save Excel: {e}")
else:
    print("❌ Tidak ada data untuk diproses.")


🚀 [3/3] MENGHITUNG BENCHMARK & MEMBUAT LAPORAN...

💾  SUKSES! Laporan Tersimpan: 'Benchmark_Per_Alat_Berat_Data_Baru2.xlsx'
    - Unit Aktif: 183
    - Unit Inaktif: 53
